## TOY PROJECT 04 - 네이버 영화 감성분석

네이버 영화 사이트의 영화 리뷰 감성을 분석합니다. 


### 1. 라이브러리 불러오기
데이터 핸들링을 위해 `pandas`,  
한국어 자연어처리를 위해 `konlpy`,  
__를 위해 `gensim` 라이브러리를 활용합니다.

In [4]:
def print_lib_ver(lib):
    print(f"{lib.__name__}: {lib.__version__}")

import numpy as np
import pandas as pd
import konlpy
import gensim

import tensorflow as tf
import matplotlib.pyplot as plt

print_lib_ver(pd)
print_lib_ver(konlpy)
print_lib_ver(gensim)
print_lib_ver(tf)

pandas: 2.0.0
konlpy: 0.6.0
gensim: 4.3.1


In [12]:
import os

def data_path(data_name):
    return f"{os.getcwd()}/data/{data_name}"

train_data = pd.read_table(data_path('ratings_train.txt'))
test_data = pd.read_table(data_path('ratings_test.txt'))

train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [13]:
from konlpy.tag import Mecab
from collections import Counter

tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def load_data(train_data, test_data, num_words=10000):
    
    # 훈련 데이터의 중복&결측 데이터 제거
    train_data.drop_duplicates(subset=['document'], inplace=True) 
    train_data = train_data.dropna(how = 'any') 
    
    # 테스트 데이터의 중복&결측 데이터 제거
    test_data.drop_duplicates(subset=['document'], inplace=True)
    test_data = test_data.dropna(how = 'any') 
    
    # 훈련 데이터 토큰화 및 불용어 제거
    X_train = []
    for sentence in train_data['document']:
        temp_X = tokenizer.morphs(sentence) 
        temp_X = [word for word in temp_X if not word in stopwords] 
        X_train.append(temp_X)

    # 테스트 데이터 토큰화 및 불용어 제거
    X_test = []
    for sentence in test_data['document']:
        temp_X = tokenizer.morphs(sentence) 
        temp_X = [word for word in temp_X if not word in stopwords]
        X_test.append(temp_X)
    
    # 사전 word_to_index 구성
    words = np.concatenate(X_train).tolist()
    counter = Counter(words)
    counter = counter.most_common(10000-4)
    vocab = ['', '', '', ''] + [key for key, _ in counter]
    word_to_index = {word:index for index, word in enumerate(vocab)}
    
    # 단어 텍스트 리스트를 사전 인덱스 스트링으로 변환
    def wordlist_to_indexlist(wordlist):
        return [word_to_index[word] if word in word_to_index else word_to_index[''] for word in wordlist]
    
    # 정제된 X_train, X_test 데이터
    X_train = list(map(wordlist_to_indexlist, X_train))
    X_test = list(map(wordlist_to_indexlist, X_test))
    
    # 각각 X_train, y_train, X_test, y_test, word_to_index → 전처리 후 데이터 반환   
    return X_train, np.array(list(train_data['label'])), X_test, np.array(list(test_data['label'])), word_to_index
    

X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data) 

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/

In [11]:
words = np.concatenate(X_train).tolist()
counter = Counter(words)
counter = counter.most_common(10000-4)
vocab = ['', '', '', ''] + [key for key, _ in counter]
word_to_index = {word:index for index, word in enumerate(vocab)}
counter

NameError: name 'word_to_index' is not defined

In [ ]:
index_to_word = {index:word for word, index in word_to_index.items()}
index_to_word

In [ ]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다. 
# 단, 모든 문장은 <BOS>로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['<BOS>']]+[word_to_index[word] if word in word_to_index else word_to_index['<UNK>'] for word in sentence.split()]

# 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<UNK>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

# 여러 개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

### 3. 모델 구성을 위한 데이터 분석 및 가공

- 데이터 셋 내부 문장 길이 분포 확인  
- 적절한 최대 문장 길이 지정  
- keras ~ pad_sequence를 활용한 패딩 추가 

### 4. 모델 구성 및 validation set 구성  

#### 4.1. CNN


#### 4.2. RNN

### 5. 모델 훈련 개시

### 6. Loss, Accuracy 그래프 시각화

### 7. 학습된 Embedding Layer 분석

### 8. 한국어 word2vec 임베딩 활용해 성능 개선